In [ ]:
import import_ipynb
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from datetime import datetime as dt
from torch.utils.data import Dataset
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler
import pickle

In [ ]:
from feeds import BackFeed,DataFeed
import utils
import models

In [ ]:
from mlstrats import ConfigFields,ModelConfig,MLConfig,MLBaseStrat,MLStrat

Host Service via Anvil

In [ ]:
# import anvil.server

In [ ]:
# DeepTrader:
# anvil.server.connect("STKYCIXUMGFV2CURGMCMOFUV-MBK5ZXFC4MCKYKOE")

In [ ]:
mlStrat=MLStrat()
mlStrat.style='seq'

In [ ]:
@anvil.server.callable
def check_entry_batch_ml0(dfD):
    global mlStrat
    # mlStrat=MLStrat()
    dfDloc={}
    decisionsD={}
    for t in dfD.keys():
        dfDloc[t]=pd.DataFrame(dfD[t])
    return MLStrat.Check(mlStrat,dfDloc)

Load data for training

In [ ]:
from genfuncs import accuracy_var, TsDS, set_label, TickDS
from labelfuncs import compute_labels
from featfuncs import feat_aug,add_addl_features_feed,add_ta_features_feed,add_sym_feature_feed

In [ ]:
DATAFILE='~/DataLocal/algo_fin_new/augdata_02-Mar-2022.csv'
df=pd.read_csv(DATAFILE)
#df=df.loc[df['Date']=='06-Dec-2021']

In [ ]:
tickers=['TCS.NS','WIPRO.NS']

In [ ]:
dataFeed=DataFeed(tickers=tickers,dfgiven=True,df=df)

In [ ]:
add_addl_features_feed(dataFeed,tickers=tickers,drop_ta=False)

In [ ]:
with open ('../../temp_data/datafeed.pickle','wb+') as f: pickle.dump(dataFeed,f)

In [ ]:
with open ('../../temp_data/datafeed.pickle','rb') as f: dataFeed=pickle.load(f)

In [ ]:
dft=compute_labels(dataFeed.df,tickers=tickers,fromFeed=True)

Training model and saving in modelConfig.pickle

In [ ]:
dft=dft.fillna(0)

In [ ]:
dates=dft['Date'].unique()

In [ ]:
dates.shape

In [ ]:
OHLC_COLS=['Open_n','High_n','Low_n','Close_n']
OHLC_ORIG=['Open','High','Low','Close']
TA_COLS=['SMA_10', 'SMA_20','VOL_SMA_20','RSI_14','BBL_5_2.0','BBM_5_2.0','BBU_5_2.0',
       'BBB_5_2.0', 'BBP_5_2.0','MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9']

In [ ]:
config=ConfigFields()

In [ ]:
tickDS=TickDS(dft,dates[0:18],dates[18:20],config,35,240,padding=40,batch_size=32)

In [ ]:
train_ds,train_xs,train_fs,train_cs,train_dates,train_tickers,train_ids,train_dfs=tickDS.create_dataset(config.tar_cols,tickers=tickers,kind='train')

In [ ]:
test_ds,test_xs,test_fs,test_cs,_,_,_,_=tickDS.create_dataset(config.tar_cols,tickers=tickers,kind='test')

In [ ]:
train_ds.fit(kind='seq')
train_ds.scale(kind='seq')
train_xs.fit(kind='flat')
train_xs.scale(kind='flat')

In [ ]:
train_cs.fit(kind='flat')
train_cs.scale(kind='flat')

In [ ]:
test_ds.scale(kind='seq',scaler=train_ds.scaler)
test_xs.scale(kind='flat',scaler=train_xs.scaler)

In [ ]:
test_cs.scale(kind='flat',scaler=train_cs.scaler)

Label 8

In [ ]:
set_label([test_ds,test_xs],8)

In [ ]:
set_label([train_ds,train_xs],8)

In [ ]:
net=models.SimpleLSTM(input_size=16,hidden_sizeL=[32,128,32],output_size=4,lr=1e-3)

In [ ]:
net,_,_=models.Train(net,train_ds,verbose=True)

In [ ]:
acc,predictedL,predsL=accuracy_var(net,[s[0] for s in train_ds],[s[1] for s in train_ds],return_probs=True,verbose=True)

In [ ]:
acc,predictedL,predsL=accuracy_var(net,[s[0] for s in test_ds],[s[1] for s in test_ds],return_probs=True,verbose=True)

In [ ]:
torch.save(net.state_dict(),'../saved_models/symLSTM8.pth')

Label 6

In [ ]:
set_label([test_ds,test_xs],6)

In [ ]:
set_label([train_ds,train_xs],6)

In [ ]:
net=models.SimpleLSTM(input_size=16,hidden_sizeL=[32,128,32],output_size=4,lr=1e-3)

In [ ]:
net,_,_=models.Train(net,train_ds,verbose=True)

In [ ]:
acc,predictedL,predsL=accuracy_var(net,[s[0] for s in train_ds],[s[1] for s in train_ds],return_probs=True,verbose=True)

In [ ]:
acc,predictedL,predsL=accuracy_var(net,[s[0] for s in test_ds],[s[1] for s in test_ds],return_probs=True,verbose=True)

In [ ]:
torch.save(net.state_dict(),'../saved_models/symLSTM6.pth')

In [ ]:
net=models.MLP(dims=[16,32,128,32,16,4],lr=1e-3,task='classification')

In [ ]:
net,_,_=models.Train(net,train_xs,verbose=True)

In [ ]:
acc,predictedL,predsL=accuracy_var(net,[s[0] for s in train_xs],[s[1] for s in train_xs],return_probs=True,verbose=True)

In [ ]:
acc,predictedL,predsL=accuracy_var(net,[s[0] for s in test_xs],[s[1] for s in test_xs],return_probs=True,verbose=True)

In [ ]:
torch.save(net.state_dict(),'../saved_models/symMLP8.pth')

In [ ]:
LSTM6=models.SimpleLSTM(input_size=16,hidden_sizeL=[32,128,32],output_size=4,lr=1e-3)

In [ ]:
LSTM6.load_state_dict(torch.load('../saved_models/symLSTM6.pth'))

In [ ]:
LSTM8=models.SimpleLSTM(input_size=16,hidden_sizeL=[32,128,32],output_size=4,lr=1e-3)

In [ ]:
LSTM8.load_state_dict(torch.load('../saved_models/symLSTM8.pth'))

In [ ]:
MLP6=models.MLP(dims=[16,32,128,32,16,4],lr=1e-3,task='classification')

In [ ]:
MLP6.load_state_dict(torch.load('../saved_models/symMLP6.pth'))

In [ ]:
MLP8=models.MLP(dims=[16,32,128,32,16,4],lr=1e-3,task='classification')

In [ ]:
MLP8.load_state_dict(torch.load('../saved_models/symMLP8.pth'))

In [ ]:
modelConfig=ModelConfig(config,[16,32,128,32,16,4],[32,128,32],MLP6,MLP8,LSTM6,LSTM8,
                       train_ds.scaler,train_cs.scaler,train_xs.scaler,tickers)

In [ ]:
modelConfig=MLConfig(config,[16,32,128,32,16,4],[32,128,32],MLP6,MLP8,LSTM6,LSTM8,
                       train_ds.scaler,train_cs.scaler,train_xs.scaler,tickers)

In [ ]:
with open('../saved_models/modelConfig.pickle','wb') as f: pickle.dump(modelConfig,f)

Experiments